# 数字媒体技术基础

## 第一讲 视频信号

## 作业1
---

将摄像头采集到的数据存成YUV文件
  - 帧率：15fps
  - 尺寸：320x240
  - 色度格式：4：2：0



### 导入有关库

- 导入视频读取处理库 `cv2`

- 导入系统库 `os`

- 导入数学库 `numpy`

- 导入二进制读写库 `struct`

In [1]:
import cv2
import os
import numpy as np
from struct import *

### 视频文件读取

读取视频文件，获取相应参数

In [2]:
V_PATH = 'video.mp4'
vcap =  cv2.VideoCapture(V_PATH)
fps = vcap.get(cv2.CAP_PROP_FPS)
size = (int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
        int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print 'fps:',fps,'size:',size

fps: 30.1884793878 size: (1280, 720)


### RGB2YUV

RGB到YUV转换，并输出到文件。

In [3]:
f = open('out_put1.yuv','wb')# 打开输出文件
WIDTH = 320
HEIGHT = 240
FPS_OUT = 15
# 变化矩阵
YUV_M = np.array([(0.299,0.587,0.114),(-0.169,-0.332,0.5),(0.5,-0.419,-0.0813)])
YUV_B = np.array([0,128,128])
FRAMES = 100 # 设置转换总帧数（未压缩文件太大）
fps_count = fps // FPS_OUT
f_count = 0
for f_num in xrange(FRAMES):
    success, frame = vcap.read()
    if f_count >= fps_count:
        f_count = 0
        continue
    else:
        f_count += 1
    if success != True:
        break
    resized = cv2.resize(frame,(WIDTH,HEIGHT))
    frame_size = int(HEIGHT * WIDTH * 1.5)
    out_put = np.zeros((frame_size),dtype = np.uint8)
    count = 0
    for i in xrange(HEIGHT):
        for j in xrange(WIDTH):
            ind = i * WIDTH + j
            temp = resized[i,j,:]
            temp[0] ,temp[2]= resized[i,j,:][2], resized[i,j,:][1]
            YUV = (np.dot(YUV_M, temp) + YUV_B)
            out_put[ind] = int(YUV[0])
            if (i % 2 == 0 and ind % 2 == 0):
                out_put[WIDTH * HEIGHT + count] = int(YUV[1])
                out_put[WIDTH * HEIGHT + WIDTH * HEIGHT/4 + count] = int(YUV[2])
                count += 1
    for b in out_put:
        f.write(pack("B", b))
f.close()
   